In [ ]:
'''
Set path to the folder of used python files
'''
import sys
sys.path.insert(0, '/Users/lynnjiang/liuGit/Pybook/Clinic-Analysis/Scripts/Data_Cleaning')

In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from Abstract import Abstract
from Diagosis import Diagnosis
from Labevents import Labevents
from Prescriptions import Prescriptions

In [3]:
from sklearn import metrics
'''
    rand_index: measures the similarity of the two assignments, ignoring permutations and with chance normalization
    Mutual Information: a function that measures the agreement of the two assignments, ignoring permutations.
    homogeneity: each cluster contains only members of a single class.
    completeness: all members of a given class are assigned to the same cluster。
'''
def cluster_metrics(pred_col, true_col):
    rand_score = metrics.adjusted_rand_score(true_col,pred_col)
    mutual_info = metrics.mutual_info_score(true_col,pred_col)
    homo_score = metrics.homogeneity_score(true_col,pred_col)
    comp_score = metrics.completeness_score(true_col,pred_col)
    return [rand_score, mutual_info, homo_score, comp_score]

metrics_columns = ["Adjusted_Rand_Index","Mutual_Information","Homogeneity", "Completeness"]

In [12]:
# selected_user_list = labevents.read_data('temp/sample_patients/PATIENTS_RAND1')
labevents.read_measurements_data(selected_user_list).head()

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SUBJECT_ID,ITEMID,VALUE,VALUENUM,LABEL


In [8]:
## Get 10 randomly selected samples from PATIENTS.csv
diagnosis = Diagnosis()
all_diagnoses = diagnosis.get_labels()
labevents = Labevents()
lab_metrics = []
for i in range(0,10):
    selected_user_list = diagnosis.create_selected_users("PATIENTS_RAND%s" % i)
    ## Transform raw data in LABEVENTS into user vectors of clinical features
    lab_user_vectors = labevents.get_uservectors(selected_user_list,labevents.read_measurements_data(selected_user_list))
    ## Split user vectors into the user list and pure vectors
    lab_user_invec, lab_user_purevec = lab_user_vectors['SUBJECT_ID'], lab_user_vectors.drop('SUBJECT_ID',1)
    ## Run cluster model
    lab_kmeans = KMeans(n_clusters=2).fit(lab_user_purevec)
    lab_pred_df = pd.DataFrame({'SUBJECT_ID':lab_user_invec,'PRED':labkmeans.labels_})
    lab_valid_df = labevents.left_join(lab_pred_df,all_diagnoses,'SUBJECT_ID')
    lab_valid_df[['TRUE']] = lab_valid_df[['LABEL']].fillna(0)
    lab_metrics.append(cluster_metrics(lab_valid_df['TRUE'],lab_valid_df['PRED']))
    

/Users/lynnjiang/liuGit/Pybook/Clinic-Analysis/Scripts/Data_Cleaning/Diagosis.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  filtered_diseass['LABEL'] = 1
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.

In [21]:
aa = Abstract()
joined_df = aa.read_data('temp/joined_user_vectors_resetindex')
lab_user_vectors = aa.read_data('temp/labtest_uservectors').sort_values(by = 'SUBJECT_ID')
pres_user_vectors = aa.read_data('temp/prestest_uservectors').sort_values(by = 'SUBJECT_ID')
diagnoses = aa.read_data('temp/diagnosis_label_4280').sort_values(by = 'SUBJECT_ID')

# joined_df.sort_values(by='SUBJECT_ID').head()
lab_user_purevectors = lab_user_vectors.drop('SUBJECT_ID', 1)
lab_user_purevectors.head()
pres_user_purevectors = pres_user_vectors.drop('SUBJECT_ID', 1)
pres_user_purevectors.head()

,AVG_DOSE_VAL_RX,AVG_DOSE_VAL_RX.1,AVG_DOSE_VAL_RX.2,AVG_DOSE_VAL_RX.3,AVG_DOSE_VAL_RX.4,AVG_DOSE_VAL_RX.5,AVG_DOSE_VAL_RX.6,AVG_DOSE_VAL_RX.7,AVG_DOSE_VAL_RX.8,AVG_DOSE_VAL_RX.9,AVG_DOSE_VAL_RX.10,AVG_DOSE_VAL_RX.11,AVG_DOSE_VAL_RX.12,AVG_DOSE_VAL_RX.13,AVG_DOSE_VAL_RX.14,AVG_DOSE_VAL_RX.15,AVG_DOSE_VAL_RX.16,AVG_DOSE_VAL_RX.17,AVG_DOSE_VAL_RX.18,AVG_DOSE_VAL_RX.19
0,0.00,0.0,177.272727,0.666667,125.000000,25.000000,9.090909,0.000000,0.0,0.000000,0.000000,0.666667,50.000000,0.0,0.937500,42.857143,6.666667,16.666667,0.750000,0.000000
1,100.00,0.0,227.500000,0.000000,350.000000,0.000000,11.666667,0.000000,0.0,50.000000,0.000000,0.000000,50.000000,0.0,0.000000,0.000000,20.000000,0.000000,1.200000,0.000000
2,260.05,125.0,133.250000,0.727273,73.076923,25.000000,11.333333,1159.090909,0.0,48.387097,1.833333,1.000000,28.928571,0.0,4.048387,250.000000,12.307692,18.181818,0.571429,318.181818
3,0.00,0.0,70.961538,0.000000,19.444444,7.142857,2.000000,0.000000,0.0,41.666667,0.071429,0.192308,0.000000,0.0,0.264706,27.419355,3.098592,8.823529,0.000000,0.000000
4,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,12.500000,0.000000,0.000000,0.000000


## LABEVENTS

In [22]:
## Group patients into 2 clusters based on their clinical features
kmeans = KMeans(n_clusters=2).fit(lab_user_purevectors)
aa.write2file(pd.DataFrame({'SUBJECT_ID':lab_user_vectors['SUBJECT_ID'],'PRED':kmeans.labels_}),\
              'lab_predicted_diagnosis')

In [23]:
lab_pred_df = pd.DataFrame({'SUBJECT_ID':lab_user_vectors['SUBJECT_ID'],'PRED':kmeans.labels_})

In [24]:
lab_valid_df = aa.left_join(lab_pred_df,diagnoses,'SUBJECT_ID')
lab_valid_df[['TRUE']] = lab_valid_df[['LABEL']].fillna(0)
lab_valid_df.head()

,SUBJECT_ID,PRED,LABEL,TRUE
0,28,0,NaN,0.0
1,34,0,1.0,1.0
2,34,0,1.0,1.0
3,36,0,NaN,0.0
4,64,0,NaN,0.0


In [25]:
np.mean(lab_valid_df['PRED']==lab_valid_df['TRUE'])

0.7190016103059581

In [26]:
lab_metrics = cluster_metrics(lab_valid_df['TRUE'],lab_valid_df['PRED'])

## Prescriptions

In [27]:
pres_user_purevectors.columns = range(pres_user_purevectors.shape[1])
pres_user_purevectors.head()
# kmeans = KMeans(n_clusters=2).fit(pres_user_purevectors)
# aa.write2file(pd.DataFrame({'SUBJECT_ID':lab_user_vectors['SUBJECT_ID'],'PRED':kmeans.labels_}),\
#               'lab_predicted_diagnosis')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.00,0.0,177.272727,0.666667,125.000000,25.000000,9.090909,0.000000,0.0,0.000000,0.000000,0.666667,50.000000,0.0,0.937500,42.857143,6.666667,16.666667,0.750000,0.000000
1,100.00,0.0,227.500000,0.000000,350.000000,0.000000,11.666667,0.000000,0.0,50.000000,0.000000,0.000000,50.000000,0.0,0.000000,0.000000,20.000000,0.000000,1.200000,0.000000
2,260.05,125.0,133.250000,0.727273,73.076923,25.000000,11.333333,1159.090909,0.0,48.387097,1.833333,1.000000,28.928571,0.0,4.048387,250.000000,12.307692,18.181818,0.571429,318.181818
3,0.00,0.0,70.961538,0.000000,19.444444,7.142857,2.000000,0.000000,0.0,41.666667,0.071429,0.192308,0.000000,0.0,0.264706,27.419355,3.098592,8.823529,0.000000,0.000000
4,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,12.500000,0.000000,0.000000,0.000000


In [28]:
## Check infinite value
# pres_user_purevectors[:20]
# np.isfinite(pres_user_purevectors.T).all()[:20]

In [29]:
pres_user_purevectors_deinfinit = pres_user_purevectors.replace([np.inf, -np.inf], 0)

In [30]:
kmeans2 = KMeans(n_clusters=2).fit(pres_user_purevectors_deinfinit)

In [31]:
## Write predicted results using prescription data into files
# aa.write2file(pd.DataFrame({'SUBJECT_ID':pres_user_vectors['SUBJECT_ID'],'PRED':kmeans2.labels_}),\
#               'pres_predicted_diagnosis')

In [32]:
pres_pred_df = pd.DataFrame({'SUBJECT_ID':pres_user_vectors['SUBJECT_ID'],'PRED':kmeans2.labels_})
pres_valid_df = aa.left_join(pres_pred_df,diagnoses,'SUBJECT_ID')
pres_valid_df[['TRUE']] = pres_valid_df[['LABEL']].fillna(0)
pres_valid_df.head()

,SUBJECT_ID,PRED,LABEL,TRUE
0,28,0,NaN,0.0
1,34,0,1.0,1.0
2,34,0,1.0,1.0
3,36,0,NaN,0.0
4,64,0,NaN,0.0


In [33]:
np.mean(pres_valid_df['PRED']==pres_valid_df['TRUE'])

0.6325242718446602

In [34]:
pres_metrics = cluster_metrics(pres_valid_df['TRUE'],pres_valid_df['PRED'])

In [35]:
## Show cluster metrics for labevents data and prescription data respectively
pd.DataFrame.from_dict({"Labevents": lab_metrics,"Prescriptions": pres_metrics}, orient='index', columns=metrics_columns)

,Adjusted_Rand_Index,Mutual_Information,Homogeneity,Completeness
Labevents,0.061235,0.008113,0.028973,0.013732
Prescriptions,-0.005138,0.000097,0.000262,0.000156


## Combination

In [ ]:
joined_df_deinf = joined_df.replace([np.inf, -np.inf], 0).drop('SUBJECT_ID', 1)
kmeans3 = KMeans(n_clusters=2).fit(joined_df_deinf)
joined_pred_df = pd.DataFrame({'SUBJECT_ID':joined_df['SUBJECT_ID'],'PRED':kmeans3.labels_})
joined_pred_df.head()

## K-fold Random Sampling